In [224]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from openpyxl.chart.label import DataLabelList
from openpyxl.chart import BarChart, Reference, PieChart, PieChart3D, Series


In [225]:
# Считываем результат
df = pd.read_excel('data/survey IT.xlsx')

In [226]:
df.shape

(3, 64)

In [227]:
# Создаем датафрейм с простыми вопросами, удаляя все колонки с /
simple_df = df.drop(columns=[column for column in df.columns if '/' in column])
simple_df.drop(columns=['ID','Время создания','Время изменения'],inplace=True)

In [228]:
simple_df.columns

Index(['Сколько вам лет?', 'Почему вы стали работать в этой сфере?',
       'Сколько человек в вашем IT-отделе вашей организации',
       'Сколько единиц компьютеров и ноутбуков (включая учебные), оргтехники(принтеры,МФУ,сканеры и т.п.) в вашей организации?',
       'Средний возраст компьютеров и ноутбуков которые вы обслуживаете',
       'Как вы оцениваете состояние сетевой инфраструктуры(Интернет, состояние ЛВС, маршрутизаторы,коммутаторы) вашей организации?',
       'Как вы оцениваете общий уровень компьютерной грамотности в вашей организации?',
       'Уровень вашей зарплаты в организации, считая все ваши ставки',
       'Насколько хорошо вы знаете UNIX-системы',
       'Сколько по вашему мнению вы должны получать за вашу работу в организации?',
       'Как сильно изменится ваше желание работать в организации на текущей должности, если вы сможете работать по свободному графику(сокращенный рабочий день, сокращенная рабочая неделя,удаленка и т.п.)?',
       'Как сильно изменится ваше

In [229]:
# Создаем список листов c одновариантными вопросами
lst_single_answer_sheets =['Возраст','Причина','Количество человек','Количество оборудования','Возраст оборудования',
                  'Сетевая инфраструктура','Компьютерная грамотность','Текущая зарплата','Знание UNIX',
                  'Желаемая зарплата','Свободный график','Поддержка проектов','Свободное время',
                  'Профессиональный рост','Идеи','Усталость','Неожиданные задачи','Частота благодарности','Предпочтения в работе',
                  'Ценность работы','Что для вас заработк','Инициатива','Поиск работы','Руководство'] 
lst_multi_answer_sheets = ['Задачи','Умения сисадмина','Языки программирования',]

In [230]:
# Создаем документ openpyxl
wb = openpyxl.Workbook()
# Создаем листы для односложных вопросов
for idx,name_sheet in enumerate(lst_single_answer_sheets):
    wb.create_sheet(title=name_sheet,index=idx+2)
    
# Создаем листы для вопросов с несколькими вариантами
for idx,name_sheet in enumerate(lst_multi_answer_sheets):
    wb.create_sheet(title=name_sheet,index=idx)



In [231]:
#Обрабатываем данные
for idx,name_sheet in enumerate(lst_single_answer_sheets):
    temp_df = simple_df.iloc[:,idx].value_counts().to_frame().reset_index()
    
    temp_df.columns = ['Возраст','Количество']
    #Высчитываем долю
    temp_df['Доля в %'] = ((temp_df['Количество'] / temp_df['Количество'].sum()).round(2) * 100)
    
    #Делаем заголовок
    wb[name_sheet]['A1'] = simple_df.columns[idx]
    for row in dataframe_to_rows(temp_df, index=False, header=True):
        wb[name_sheet].append(row)
    
      # Рисуем диаграмму
    #выбираем значения
    labels = Reference(wb[name_sheet],min_col=1,min_row=3,max_row=temp_df.shape[0]+2)
    values = Reference(wb[name_sheet],min_col=3,min_row=2,max_row=temp_df.shape[0]+2)
    # Создаем объект диаграммы
    pie_chart = PieChart()
    pie_chart.add_data(values,titles_from_data=True)
    pie_chart.set_categories(labels)
    
    series = Series(temp_df['Доля в %'], title='Series 1')
    pie_chart.append(series)
    pie_chart.title = f"{name_sheet}"
    #Привязываем диаграмму к ячейке А15
    wb[name_sheet].add_chart(pie_chart,'A15')
    # Определяем размеры диаграммы

    pie_chart.width =20
    pie_chart.height = 5
    
    wb[name_sheet].column_dimensions['A'].width = 40
    wb[name_sheet].column_dimensions['B'].width = 15
 
    


TypeError: expected string or bytes-like object

In [ ]:
#Удаялем лишний лист
del wb['Sheet']
current_time = time.strftime('%H_%M_%S')
wb.save(f'Итоги опроса от {current_time}.xlsx')